## SCE4104: Practical Applications of Computer Vision Assignment
## Part 2: Results Analysis

How to use:

Set the appropriate paths in cell 2 to the csv files required.
Restart and run all code to generate results.

Import the necessary libraries

In [1]:
import numpy as np
import pandas as pd
import math
import csv
from sklearn.metrics import mean_absolute_error

Set paths for csv files

In [2]:
path_src = 'E:/Engineering 4th Year/SCE4104 Practical Applications of Computer Vision/Assignment/Results_FINAL/vid3.csv'
path_dst = 'E:/Engineering 4th Year/SCE4104 Practical Applications of Computer Vision/Assignment/Results_FINAL/Results_TruePositives/vid3_TP.csv'
path_cmp = 'E:/Engineering 4th Year/SCE4104 Practical Applications of Computer Vision/Assignment/Annotation Files/cam-3.csv'

Load data as numpy array

In [3]:
data_src = pd.DataFrame.to_numpy(pd.read_csv(path_src))
data_cmp = pd.DataFrame.to_numpy(pd.read_csv(path_cmp))

Ensure that the data is consistent

In [4]:
rows_src = len(data_src)
cols_src = len(data_src[0])

rows_cmp = len(data_cmp)
cols_cmp = len(data_cmp[0])

if(rows_src != rows_cmp) or (cols_src != cols_cmp):
    print('Number of rows/columns of source file is not equal to number of rows/columns in ground truth')
    print('Check and fix error before continuing')
else:
    print('Same number of rows/columns in source file and ground truth')
    print('Carry on')

Same number of rows/columns in source file and ground truth
Carry on


Remove false positives by thresholding objects more than 50 pixels away from the ground truth and removing detected objects which do not have a corresponding ground truth.
Note also instances of true positives, true negatives, and false positives for statistical use at a later stage.

In [5]:
data_dst = np.zeros((0,cols_src))
tp_src = 0
tn_src = 0
fp_src = 0

for i in range(rows_src):
    frame = data_src[i][0]
    x_src = data_src[i][1]
    x_cmp = data_cmp[i][1]
    y_src = data_src[i][2]
    y_cmp = data_cmp[i][2]
    
    if x_cmp == ' -' or x_cmp == '-':
        x_cmp_exist = True
    else:
        x_cmp_exist = False
        
    if x_src == '-' and x_cmp_exist == True:
        tn_src = tn_src+1
        
    if x_src != '-' and x_src!= ' -' and x_cmp_exist == False:
        x_diff = abs(int(x_src) - int(x_cmp))
        y_diff = abs(int(y_src) - int(y_cmp))
        if x_diff > 50 or y_diff > 50:
            newdata = [frame,'-','-']
            data_dst = np.vstack([data_dst,newdata])
            fp_src = fp_src + 1
        else:
            newdata = [frame,x_src,y_src]
            data_dst = np.vstack([data_dst,newdata])
            tp_src = tp_src + 1
    else:
        newdata = [frame,'-','-']
        data_dst = np.vstack([data_dst,newdata])
        
print(data_dst, tp_src, tn_src)

[['1' '-' '-']
 ['2' '-' '-']
 ['3' '-' '-']
 ...
 ['2997' '271' '389']
 ['2998' '-' '-']
 ['2999' '-' '-']] 165 1275


Save the new data to the csv file to keep record

In [6]:
pd.DataFrame(data_dst).to_csv(path_dst, header=['Frame No.', 'x', 'y'], index=None)

Ensure the data structure is correct

In [7]:
# data_cmp - ground truth
# data_dst - true positives

rows_dst = len(data_dst)
cols_dst = len(data_dst[0])

rows_cmp = len(data_cmp)
cols_cmp = len(data_cmp[0])

if(rows_dst != rows_cmp) or (cols_dst != cols_cmp):
    print('Number of rows/columns of true positives is not equal to number of rows/columns in ground truth')
    print('Check and fix error before continuing')
else:
    print('Same number of rows/columns in true positives and ground truth')
    print('Carry on')

Same number of rows/columns in true positives and ground truth
Carry on


Instantiate new sets of arrays

In [8]:
x_dst = []
y_dst = []
x_cmp = []
y_cmp = []

Compute the mean absolute error, for each true positive location for the ball

In [9]:
# Replace all instances of '-' with 0
x_dst = data_dst[:,1]
x_dst[:] = [0 if x=='-' else x for x in x_dst] 
y_dst = data_dst[:,2]
y_dst[:] = [0 if y=='-' else y for y in y_dst]
x_cmp = data_cmp[:,1]
x_cmp[:] = [0 if x=='-' else x for x in x_cmp]
x_cmp[:] = [0 if x==' -' else x for x in x_cmp]
y_cmp = data_cmp[:,2]
y_cmp[:] = [0 if y=='-' else y for y in y_cmp]
y_cmp[:] = [0 if y==' -' else y for y in y_cmp]


# Number of true positives and true negatives in ground truth
tp_cmp = len(x_cmp[x_cmp != 0])
tn_cmp = len(x_cmp) - tp_cmp

# Ignore ground truths which correspond to false negatives by detections
for i in range(len(x_dst)):
    if x_dst[i] == '0':
        x_cmp[i] = 0
        y_cmp[i] = 0
        
# Remove zero elements to compute over true positives only
x_dst_f = np.array(x_dst[x_dst != '0'], dtype=float)
x_cmp_f = np.array(x_cmp[x_cmp != 0], dtype=float)
y_dst_f = np.array(y_dst[y_dst != '0'], dtype=float)
y_cmp_f = np.array(y_cmp[y_cmp != 0], dtype=float)

In [10]:
y_mae = mean_absolute_error(y_cmp_f, y_dst_f)
x_mae = mean_absolute_error(x_cmp_f, x_dst_f)

In [11]:
sens = tp_src*100/tp_cmp

In [12]:
spec = tn_src*100/tn_cmp

In [13]:
print("Mean Absolute Error for x co-ordinates: ", x_mae)
print("Mean Absolute Error for y co-ordinates: ", y_mae)
print("Sensitivity (TP rate): ", sens,"%")
print("Specificity (TN rate): ", spec,"%")

Mean Absolute Error for x co-ordinates:  3.3636363636363638
Mean Absolute Error for y co-ordinates:  4.375757575757576
Sensitivity (TP rate):  10.529674537332482 %
Specificity (TN rate):  89.03631284916202 %
